In [1]:
!pip install gradio youtube_transcript_api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.6 MB 4.3 MB/s 
     |████████████████████████████████| 278 kB 23.8 MB/s 
     |████████████████████████████████| 84 kB 842 kB/s 
     |████████████████████████████████| 55 kB 1.3 MB/s 
     |████████████████████████████████| 54 kB 732 kB/s 
     |████████████████████████████████| 106 kB 5.2 MB/s 
     |████████████████████████████████| 2.3 MB 25.6 MB/s 
     |████████████████████████████████| 84 kB 1.2 MB/s 
     |████████████████████████████████| 56 kB 657 kB/s 
     |████████████████████████████████| 213 kB 39.7 MB/s 
     |████████████████████████████████| 64 kB 812 kB/s 
     |████████████████████████████████| 80 kB 5.0 MB/s 
     |████████████████████████████████| 68 kB 5.9 MB/s 
     |████████████████████████████████| 68 kB 6.0 MB/s 
     |████████████████████████████████| 68 kB 6.0 MB/s 
     |████████████████████████████████| 46 kB 4.1 MB

In [8]:
import gradio
from youtube_transcript_api import YouTubeTranscriptApi


def yt2transcript(video_id):
  print(f"\n\nid: {video_id}")

  # data looks like [{'text': 'hey friends welcome to one little coder', 'start': 0.84, 'duration': 4.38}, ...]
  data = YouTubeTranscriptApi.get_transcript(video_id)
  transcript = ' '.join([x['text'] for x in data])

  # TODO if there is no transcript (how likely is this?), run through whisper-large on hf (but 30k free characters per month)

  return transcript

#demo = gr.Interface(fn=yt2transcript, inputs=["text"], outputs=["text"])
                    #article="\n".join([f"- {k}: " + v.replace("\n"," ") for k,v in instructions.items()]))
#demo.launch(debug=True, share=True)   

In [9]:
video_id = "mbn6yLcoyzo"
print(yt2transcript(video_id))



id: mbn6yLcoyzo
hey friends welcome to one little coder if you ever wanted to build a machine learning application you would know that sometimes you have to wrap this entire machine learning application as an API so for your application you need front-end and back-end sometimes your front end could be not python for example it could be react.js it could be angularjsr it could be any JavaScript framework that you want to use while the backend being powered by Python and most of the time people actually use apis for it the APA is communicating to the front end and making calls to the server and this is how the full stack application is going to look like but traditionally if you wanted to use an API you need to host it on a server and then use something like Fast API and then get the APA so this is what we have been doing all the while but today or probably yesterday night radio announced a new feature where you can use gradu as an API this has been there for a while we probably would 

In [ ]:
def ingest_transcript(transcript):
  # TODO langchain and...elastic maybe? cognosis ai? qdrant/weaviate/pinecone?